In [1]:
import pandas as pd
import requests
import matplotlib.pyplot as plt

In [3]:
url = f"https://api.binance.com/api/v3/depth"
params = {
    "symbol": "SOLUSDT",
    "limit": 10
}

orderbook = requests.get(url, params=params).json()
orderbook

{'lastUpdateId': 26451373426,
 'bids': [['136.26000000', '386.47500000'],
  ['136.25000000', '164.09000000'],
  ['136.24000000', '236.30200000'],
  ['136.23000000', '430.50500000'],
  ['136.22000000', '656.64700000'],
  ['136.21000000', '202.95400000'],
  ['136.20000000', '298.92200000'],
  ['136.19000000', '324.43700000'],
  ['136.18000000', '560.83000000'],
  ['136.17000000', '182.24300000']],
 'asks': [['136.27000000', '88.48200000'],
  ['136.28000000', '198.31800000'],
  ['136.29000000', '195.92600000'],
  ['136.30000000', '295.85300000'],
  ['136.31000000', '421.98300000'],
  ['136.32000000', '546.31100000'],
  ['136.33000000', '1490.94600000'],
  ['136.34000000', '319.22800000'],
  ['136.35000000', '496.69000000'],
  ['136.36000000', '1211.08300000']]}

In [10]:
ask_levels = pd.DataFrame(orderbook['asks'], columns=['price', 'quantity'], dtype=float)
ask_levels['side'] = 'ask'
ask_levels = ask_levels.sort_values('price')
ask_levels

,price,quantity,side
0,136.27,88.482,ask
1,136.28,198.318,ask
2,136.29,195.926,ask
3,136.30,295.853,ask
4,136.31,421.983,ask
5,136.32,546.311,ask
6,136.33,1490.946,ask
7,136.34,319.228,ask
8,136.35,496.690,ask
9,136.36,1211.083,ask


In [9]:
bid_levels = pd.DataFrame(orderbook['bids'], columns=['price', 'quantity'], dtype=float)
bid_levels['side'] = 'bid'
bid_levels = bid_levels.sort_values('price', ascending=False)
bid_levels

,price,quantity,side
0,136.26,386.475,bid
1,136.25,164.090,bid
2,136.24,236.302,bid
3,136.23,430.505,bid
4,136.22,656.647,bid
5,136.21,202.954,bid
6,136.20,298.922,bid
7,136.19,324.437,bid
8,136.18,560.830,bid
9,136.17,182.243,bid


In [13]:
# calculate orderbook mid price
def calculate_mid_price(bid_levels, ask_levels):
    mid_price = (bid_levels['price'].iloc[0] + ask_levels['price'].iloc[0]) / 2
    return mid_price